# ***Diabetes Prediction — Logistic, Decision Tree & Random Forest***
 
## **This notebook walks through:**
 1. **EDA** (Exploratory Data Analysis)
 2. **Preprocessing** (Cleaning, Encoding, Scaling, Balancing)
 3. **Training** (3 models: Logistic Regression, Decision Tree, Random Forest)
 4. **Evaluation** (Confusion Matrix, Classification Report, ROC & AUC curves)**


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import f1_score, confusion_matrix, classification_report, accuracy_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from imblearn.over_sampling import  SMOTE

Configured train path: /kaggle/input/diabetes/train_Data.csv
Configured test path : /kaggle/input/diabetes/test_Data.csv


In [ ]:
train_df = pd.read_csv("/kaggle/input/diabetes/train_Data.csv")
test_df  = pd.read_csv("/kaggle/input/diabetes/test_Data.csv")

print("Train shape:", train_df.shape)
print("Test shape :", test_df.shape)

train_df.head()

Train shape: (80000, 10)
Test shape : (20000, 10)


,Unnamed: 0,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,0,Female,80.0,0,1,never,25.19,6.6,140,0
1,1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,2,Male,28.0,0,0,never,27.32,5.7,158,0
3,3,Female,36.0,0,0,current,23.45,5.0,155,0
4,4,Male,76.0,1,1,current,20.14,4.8,155,0


## ***EDA***

In [ ]:
print("Data Info:\n", train_df.info())
print("Summary Statistics:\n", train_df.describe(include="all").T)
print("Missing Values:\n", train_df.isna().sum())
print("Duplicate Rows:\n", train_df.duplicated().sum())

In [ ]:
ax = sns.countplot(x="diabetes", data=train_df, stat="percent", palette="Set2")
for bar in ax.containers:
    ax.bar_label(bar, fmt="%.1f%%")
plt.title("Distribution of Diabetes Target")
plt.ylabel("Percentage %")
plt.show()

The dataset is imbalanced, this will require balancing during preprocessing.

### *Categorical Variables Distribution*

In [ ]:
cat_cols = ["gender", "smoking_history"]

fig, axes = plt.subplots(1, len(cat_cols), figsize=(12, 4))
for i, col in enumerate(cat_cols):
    sns.countplot(x=col, data=train_df, ax=axes[i], palette="Set2")
    axes[i].set_title(f"{col} distribution")
    axes[i].tick_params(axis="x", rotation=45)
plt.tight_layout()
plt.show()

### *Numerical Variables Distribution*

In [ ]:
num_cols = ["age", "bmi", "HbA1c_level", "blood_glucose_level"]

train_df[num_cols].hist(bins=30, figsize=(10, 8), color="skyblue")
plt.suptitle("Histograms of Numerical Features", y=1.02)
plt.show()

### *Correlation Heatmap*

In [ ]:
corr = train_df[num_cols + ["diabetes"]].corr()
plt.figure(figsize=(8, 6))
sns.heatmap(corr, annot=True, cmap="coolwarm", center=0)
plt.title("Correlation Heatmap")
plt.show()

### *Checking for Outliers*

In [ ]:
# Outlier Detection (IQR method)
def detect_outliers_iqr(data: pd.Series):
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return data[(data < lower_bound) | (data > upper_bound)]

# Visualizing outliers per numerical column
numeric_cols = train_df.select_dtypes(include=np.number).columns

for col in numeric_cols:
    plt.figure(figsize=(6, 3))
    sns.boxplot(x=train_df[col])
    plt.title(f"Outliers in {col}")
    plt.show()

    outliers = detect_outliers_iqr(train_df[col])
    print(f"{col}: {len(outliers)} outliers")


## ***Preprocessing***

Creating function for each preprocessing step in order to optimize the code when applying each step for the train data and test data.

In [ ]:
def clean(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # Remove duplicates
    df = df.drop_duplicates()

    # Fill nulls in categorical
    df["gender"] = df["gender"].fillna("Unknown").astype(str)
    df["smoking_history"] = df["smoking_history"].fillna("Unknown").astype(str)

    # Fill nulls in numeric
    for col in ["age", "bmi", "HbA1c_level", "blood_glucose_level"]:
        df[col] = pd.to_numeric(df[col], errors="coerce")
        df[col] = df[col].fillna(df[col].median())

    # Clean binary columns
    for col in ["hypertension", "heart_disease", "diabetes"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0).astype(int)

    # Outlier capping (IQR)
    def cap_outliers(series: pd.Series) -> pd.Series:
        Q1, Q3 = series.quantile([0.25, 0.75])
        IQR = Q3 - Q1
        low, high = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR
        return np.clip(series, low, high)

    for col in ["age", "bmi", "HbA1c_level", "blood_glucose_level"]:
        df[col] = cap_outliers(df[col])

    return df

train_df = clean(train_df)
test_df = clean(test_df)

#### ***Feature Scaling & Categorical Columns encoding***

In [ ]:
FEATURES = ["gender","age","hypertension","heart_disease","smoking_history","bmi","HbA1c_level","blood_glucose_level"]
TARGET = "diabetes"

X_train, y_train = train_df[FEATURES], train_df[TARGET]
X_test, y_test   = test_df[FEATURES], test_df[TARGET]

categorical_features = ["gender", "smoking_history"]
numeric_features = ["age","hypertension","heart_disease","bmi","HbA1c_level","blood_glucose_level"]

preprocessor = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
    ("num", StandardScaler(), numeric_features)
])

X_train_proc = preprocessor.fit_transform(X_train)
X_test_proc  = preprocessor.transform(X_test)

### ***Data Oversampling***

In [ ]:
sampler = SMOTE(random_state=42)
X_res, y_res = sampler.fit_resample(X_train_proc, y_train)
print("Original dataset shape:", y_train.value_counts().to_dict())
print("Resampled dataset shape:", y_res.value_counts().to_dict())

## **Model Building, Training, and Evaluation**

In [ ]:
# Helper function to evaluate model directly (for code optimization purposes)
def evaluate_model(model, X_test, y_test, model_name="Model"):
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else None

    # --- Print metrics ---
    print(f"\n📌 {model_name} Evaluation:")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("F1-score:", f1_score(y_test, y_pred, average='macro'))

    if y_prob is not None:
        print("ROC-AUC:", roc_auc_score(y_test, y_prob))
    print("\nClassification Report:\n", classification_report(y_test, y_pred))

    # --- Confusion Matrix ---
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(5, 4))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False,
                xticklabels=["No Diabetes", "Diabetes"],
                yticklabels=["No Diabetes", "Diabetes"])
    plt.title(f"{model_name} - Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.show()

## ***Logistic Regression***

In [ ]:
log_reg = LogisticRegression(max_iter=1000, random_state=42)
log_reg.fit(X_res, y_res)
evaluate_model(log_reg, X_test_proc, y_test, "Logistic Regression")

## ***Decision Tree***

In [ ]:
tree = DecisionTreeClassifier(random_state=42)
tree.fit(X_res, y_res)
evaluate_model(tree, X_test_proc, y_test, "Decision Tree")


## **Random Forest**

In [ ]:
forest = RandomForestClassifier(n_estimators=100, random_state=42)
forest.fit(X_res, y_res)
evaluate_model(forest, X_test_proc, y_test, "Random Forest")